#Setup

In [1]:
TRAIN_MODEL = True

In [2]:
# Janky code to do different setup when run in a Colab notebook vs VSCode
DEVELOPMENT_MODE = True
try:
    import google.colab
    IN_COLAB = True
    print("Running as a Colab notebook")
    %pip install git+https://github.com/neelnanda-io/TransformerLens.git@new-demo
    %pip install circuitsvis

    # PySvelte is an unmaintained visualization library, use it as a backup if circuitsvis isn't working
    # # Install another version of node that makes PySvelte work way faster
    # !curl -fsSL https://deb.nodesource.com/setup_16.x | sudo -E bash -; sudo apt-get install -y nodejs
    # %pip install git+https://github.com/neelnanda-io/PySvelte.git
except:
    IN_COLAB = False
    print("Running as a Jupyter notebook - intended for development only!")
    from IPython import get_ipython

    ipython = get_ipython()
    # Code to automatically update the HookedTransformer code as its edited without restarting the kernel
    ipython.magic("load_ext autoreload")
    ipython.magic("autoreload 2")

Running as a Colab notebook
  Cloning https://github.com/neelnanda-io/TransformerLens.git (to revision new-demo) to /tmp/pip-req-build-guoo0of9
  Running command git clone --filter=blob:none --quiet https://github.com/neelnanda-io/TransformerLens.git /tmp/pip-req-build-guoo0of9
  Running command git checkout -b new-demo --track origin/new-demo
  Switched to a new branch 'new-demo'
  Branch 'new-demo' set up to track remote branch 'new-demo' from 'origin'.
  Resolved https://github.com/neelnanda-io/TransformerLens.git to commit 25879ea6d969e8e9c962957bc945be42de522b4a
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 237.5/237.5 kB 30.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 887.

In [3]:
# Plotly needs a different renderer for VSCode/Notebooks vs Colab argh
import plotly.io as pio
if IN_COLAB or not DEVELOPMENT_MODE:
    pio.renderers.default = "colab"
else:
    pio.renderers.default = "notebook_connected"
print(f"Using renderer: {pio.renderers.default}")

Using renderer: colab


In [4]:
pio.templates['plotly'].layout.xaxis.title.font.size = 20
pio.templates['plotly'].layout.yaxis.title.font.size = 20
pio.templates['plotly'].layout.title.font.size = 30

In [5]:
# Import stuff
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import csv
import pandas as pd
import einops
from fancy_einsum import einsum
import tqdm.auto as tqdm
import random
from pathlib import Path
#import plotly.express as px
from torch.utils.data import DataLoader

#from torchtyping import TensorType as TT
from typing import List, Union, Optional
from functools import partial
import copy

import itertools
from transformers import AutoModelForCausalLM, AutoConfig, AutoTokenizer
import dataclasses
import datasets
import os
import torch
import glob
import torch
from torch.utils.data import Dataset, DataLoader
from IPython.display import HTML

Install, and log into the Weight&Bias platform

In [7]:
!pip install wandb
import wandb
wandb.login()

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [9]:
import transformer_lens
import transformer_lens.utils as utils
from transformer_lens.hook_points import (
    HookedRootModule,
    HookPoint,
)  # Hooking utilities
from transformer_lens import HookedTransformer, HookedTransformerConfig, FactoredMatrix, ActivationCache

In [10]:
def imshow(tensor, renderer=None, xaxis="", yaxis="", **kwargs):
    px.imshow(utils.to_numpy(tensor), color_continuous_midpoint=0.0, color_continuous_scale="RdBu", labels={"x":xaxis, "y":yaxis}, **kwargs).show(renderer)

def line(tensor, renderer=None, xaxis="", yaxis="", **kwargs):
    px.line(utils.to_numpy(tensor), labels={"x":xaxis, "y":yaxis}, **kwargs).show(renderer)

def scatter(x, y, xaxis="", yaxis="", caxis="", renderer=None, **kwargs):
    x = utils.to_numpy(x)
    y = utils.to_numpy(y)
    px.scatter(y=y, x=x, labels={"x":xaxis, "y":yaxis, "color":caxis}, **kwargs).show(renderer)

In [11]:
# where we save the model
PTH_LOCATION = "/content/sample_data"

#Parking problem

We design a first **problem**, the *parking problem*:

Given a parking lot with n spaces, let's assume there are k cars. Each of the cars is placed in a space, the cars can only move to the left, determine if there is a parking configuration where all the cars are parked in a space that is unique for each car.

We give an algorithmic way to solve the Parking Problem; and we construct string sequences which describe the problem to our Transformers

In [12]:
def parking(L,k,n):
  S=0
  for i in range(n):
    a=L.count(i)
    #print(a)
    S+=-a+1
    #print(S)
    if S<0:
      return False
  return True

Now, we are generating sentences illustrating the problem, and with labels indicaing if there is a parking configuration or not

Generation of the list of configuration. We choose the number of parking spaces to go from 2 to N_max-1, then we add a number of cars which is from 2 to N_max+3 in order to have as much as sucess as failure of parking configuration

In [20]:
#Parameters
N_max=10
inputs_txt=[]
inputs_sorted_txt=[]
label=[]
c=0

In [25]:
for i in range (2,N_max):
  for k in range(2,N_max+4):
    c+=1

    #######Generate a random Configuration

    L = [random.randint(0, i-1) for _ in range(k)]+[-1]*(N_max+4-k-1)
    #######Compute if it is a parking problem

    Parking=parking(L,k,i)
    label.append(Parking)

    ####### Inputs text with a random list

    txt='There are '+str(k)+' cars, and there are '+str(i)+' parking lots.'
    for j in range(k-1):
      if L[j]>=0:
        txt=txt+' The car '+str(j)+' is in position '+str(L[j])+','
    txt=txt+' The car '+str(k)+' is in position '+str(L[k-1])+','
    txt=txt+' The cars can only move in a smaller position. Is it possible to find a configuration where all the cars are in a different position?'
    inputs_txt.append(txt)


    ######optional if you want to add this data on your storage
    #if Parking==1:
    #    data = {"config": [txt], "Parking": [Parking]}
    #    df = pd.DataFrame(data)
    #    df.to_csv('Parking'+str(c)+'.csv',index= False)
    #
    #else:
    #    with open('/Users/timothechambery/CRAFT_LLM/fine_tuning/Grokking/parking_dir/NoParking/NoParking'+str(c), 'w', newline='') as file:
    #      writer = csv.writer(file, delimiter=';')
    #      writer.writerows([[txt],[Parking]])

    ######Inputs text with a sorted list

    #L.sort()
    #txt='There are '+str(k)+' cars, and there are '+str(i)+' parking lots.'
    #for j in range(k-1):
    #  if L[j]>=0:
    #    txt=txt+' The car '+str(j)+' is in position '+str(L[j])+','
    #txt=txt+' The car '+str(k)+' is in position '+str(L[k-1])+','
    #txt=txt+' The cars can only move in a smaller position. Is it possible to find a configuration where all the cars are in a different position?'
    #Train_inputs_sorted_txt.append(txt)




#Train_inputs_L=np.array(Train_inputs_L)
#Train_inputs_L=torch.tensor(Train_inputs_L)
#Train_inputs_txt=np.array(Train_inputs_txt)
#Train_inputs_sorted_txt=np.array(Train_inputs_sorted_txt)
#dataset_txt=
#dataset = torch.stack([Train_inputs_L],dim=1).cuda()#,Train_inputs_k , Train_inputs_n], dim=1).cuda()
#labels=torch.tensor(Train_label)
#dataset = torch.utils.data.TensorDataset(dataset, labels)

We shuffle the data in order to have a dataset composed of Training data and of Test data. We want that fraction_{test} % of the dataset composed of Test datafr

In [81]:
frac_train=0.7

data=[[inputs_txt[i],label[i]] for i in range (c)]

random.shuffle(data)


indice=int(frac_train*c)

feat_train=[ data[i][0] for i in range(indice)]
labels_train=[data[i][1] for i in range(indice)]
feat_test=[ data[i][0] for i in range(indice,c)]
labels_test=[ data[i][1] for i in range(indice,c)]

We transform our string sequence in tokens, using an appropriate tokenizer

# Dataset creation for GPT2Small

In [83]:
feat_train_gpt2s=[gpt2s.to_tokens(x) for x in feat_train]
feat_test_gpt2s=[gpt2s.to_tokens(x) for x in feat_test]

Construct a dataset. For this, we construct our custom dataset

Here we can use the dataloader to have our data batched, and parallelized.

In [84]:
dtype=torch.float

datasetfeat_train_gpt2s = [[f.type(dtype),torch.tensor(l).type(torch.long)] for (f,l) in zip(feat_train_gpt2s,labels_train)]
datasetfeat_train_gpt2s = [(inputs.reshape(-1), classes) for [inputs,classes] in datasetfeat_train_gpt2s]

datasetfeat_test_gpt2s = [[f.type(dtype),torch.tensor(l).type(torch.long)] for (f,l) in zip(feat_test_gpt2s,labels_test)]
datasetfeat_test_gpt2s = [(inputs.reshape(-1), classes) for [inputs,classes] in datasetfeat_test_gpt2s]

In [85]:
train_loader_gpt2s = torch.utils.data.DataLoader(datasetfeat_train_gpt2s, batch_size=128, shuffle=True)
test_loader_gpt2s  = torch.utils.data.DataLoader(datasetfeat_test_gpt2s, batch_size=64, shuffle=True)

#Dataset creation for toy model transformers

In general, to construct the right dataset for your appropriate transformer architecture, you juste need to import your model before, and to replace the name of your model in **gpt2s**

DEFINITION



We define the toy models Transformers that we want to train

In [86]:
n=10
cfg = HookedTransformerConfig(
    n_layers = 1,
    n_heads = 4,
    d_model = 128,
    d_head = 32,
    d_mlp = 512,
    act_fn = "relu",
    normalization_type=None,
    d_vocab=n+1,
    d_vocab_out=n,
    n_ctx=32,
    init_weights=True,
    device="cuda",
    seed = 999,
)

In [88]:
#feat_train_cfg=[cfg.to_tokens(x) for x in feat_train]
#feat_test_cfg=[cfg.to_tokens(x) for x in feat_test]

In [89]:
#dtype=torch.float

#datasetfeat_train_cfg = [[f.type(dtype),torch.tensor(l).type(torch.long)] for (f,l) in zip(feat_train_cfg,labels_train)]
#datasetfeat_train_cfg = [(inputs.reshape(-1), classes) for [inputs,classes] in datasetfeat_train_cfg]

#datasetfeat_test_cfg = [[f.type(dtype),torch.tensor(l).type(torch.long)] for (f,l) in zip(feat_test_cfg,labels_test)]
#datasetfeat_test_cfg = [(inputs.reshape(-1), classes) for [inputs,classes] in datasetfeat_test_cfg]

#TRAINING STEP of our first toy model

In [90]:
epochs=10
wd=0.01
lr=1e-4
batch_size=32
betas=(0.9, 0.999)

Choice of Loss and optimizer

In [ ]:
optimizer = torch.optim.AdamW(cfg.parameters(), lr=lr, weight_decay=wd, betas=betas)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=epochs, eta_min=0)
loss_fn = torch.nn.CrossEntropyLoss()

If you want to disable the bias

In [ ]:
for name, param in cfg.named_parameters():
    if "b_" in name:
        param.requires_grad = False


Steps of training

In [ ]:
import torch
import torch.nn as nn
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import scipy.integrate as integrate
import scipy.optimize as optimize
import scipy.stats as stats
from numpy.random import random

# generate random input data
x = random((30,2))

learning_rate = 1e-2

# generate labels corresponding to input data x
y = np.dot(x, [2., -3.]) + 1.
w_source = np.array([2., -3.])
b_source  = np.array([1.])

# randomly initialize learnable weights and bias
w_init = random(2)
b_init = random(1)

w = w_init
b = b_init
print("initial values of the parameters:", w, b )

dtype = torch.FloatTensor
# dtype = torch.cuda.FloatTensor # Uncomment this to run on GPU

x_t = torch.from_numpy(x).type(dtype)
#ici on rajoute un unsqueeze car le module MMSE a besoin de ce dimensionnement pour fonctionner
y_t = torch.from_numpy(y).type(dtype).unsqueeze(1)

w_init_t = torch.from_numpy(w_init).type(dtype)
b_init_t = torch.from_numpy(b_init).type(dtype)

w_t = w_init_t.clone()
w_t.unsqueeze_(1)
b_t = b_init_t.clone()
b_t.unsqueeze_(1)
print("initial values of the parameters:", w_t, b_t )

#definition du modele
model = torch.nn.Sequential(
    torch.nn.Linear(2, 1),
)

for m in model.children():
    m.weight.data = w_init_t.clone().unsqueeze(0)
    m.bias.data = b_init_t.clone()

#definition d'une loss

loss_fn = torch.nn.MSELoss(reduction='sum')

#mettre le modèle en entrainement
model.train()

#definir un optimiseur
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)


for epoch in range(1000):
    #appliquer le reseau de neurones sur les données
    y_pred = model(x_t)
    #calculer la loss
    loss = loss_fn(y_pred, y_t)
    print("progress:", "epoch:", epoch, "loss",loss.item())
    # Zero gradients, perform a backward pass, and update the weights.
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()


# After training
print("estimation of the parameters:")
for param in model.parameters():
    print(param)

train_losses = []
test_losses = []
model_checkpoints = []
checkpoint_epochs = []
if TRAIN_MODEL:
    for epoch in tqdm.tqdm(range(num_epochs)):
        train_logits = model(train_data)
        train_loss = loss_fn(train_logits, train_labels)
        train_loss.backward()
        train_losses.append(train_loss.item())

        optimizer.step()
        optimizer.zero_grad()

        with torch.inference_mode():
            test_logits = model(test_data)
            test_loss = loss_fn(test_logits, test_labels)
            test_losses.append(test_loss.item())

        if ((epoch+1)%checkpoint_every)==0:
            checkpoint_epochs.append(epoch)
            model_checkpoints.append(copy.deepcopy(model.state_dict()))
            print(f"Epoch {epoch} Train Loss {train_loss.item()} Test Loss {test_loss.item()}")

In [ ]:
torch.save(
    {
        "model":model.state_dict(),
        "config": model.cfg,
        "checkpoints": model_checkpoints,
        "checkpoint_epochs": checkpoint_epochs,
        "test_losses": test_losses,
        "train_losses": train_losses,
        "train_indices": train_indices,
        "test_indices": test_indices,
    },
    PTH_LOCATION)

In [ ]:
if not TRAIN_MODEL:
    cached_data = torch.load(PTH_LOCATION)
    model.load_state_dict(cached_data['model'])
    model_checkpoints = cached_data["checkpoints"]
    checkpoint_epochs = cached_data["checkpoint_epochs"]
    test_losses = cached_data['test_losses']
    train_losses = cached_data['train_losses']
    train_indices = cached_data["train_indices"]
    test_indices = cached_data["test_indices"]

#Fine-tunning of the GPT2 small

We first modify the last two layers in order that the Transformers learns how to answer the classification task which is the parking problem.

To that purpose, we propose to add a linear layer with a softmax activation function.

In [59]:
gpt2s = HookedTransformer.from_pretrained(
    "gpt2-small",
    center_unembed=True,
    center_writing_weights=True,
    fold_ln=True,
    refactor_factored_attn_matrices=True,
)

Using pad_token, but it is not set yet.


Loaded pretrained model gpt2-small into HookedTransformer


In [ ]:
num_classes = 2
model.classifier[1] = torch.nn.Linear(1280, num_classes)

In [ ]:
for param in model.parameters():
    param.requires_grad = False

for param in model.classifier.parameters():
    param.requires_grad = True

In [ ]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.classifier.parameters(), lr=0.001, momentum=0.9)

Training of GPT2small on the classification task

Plots showing that it groks